# Calculate Mean

Note that the mean, standard deviation, and max of a value is calculated in the defined order, that is, we first have 300 of wavelength 0, then 300 of wavelength 1, etc, all squashed into 1D ahead of time (just like `np.flatten` does, **unless `np.flatten` do something out of what I am expecting?** And we are expecting that with `np.append`, we will get the flatten on the first array first, then append whatever at the back. This is the order of how the file is defined. 

In [6]:
storage_mean = "./mean.txt"
storage_std = "./std.txt"
storage_mod = "./mod_abs.txt"

In [2]:
PROJECT_ID = "sunlit-analyst-309609"
%env GCLOUD_PROJECT=$PROJECT_ID
%load_ext google.cloud.bigquery

env: GCLOUD_PROJECT=sunlit-analyst-309609


In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tempfile

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

from google.cloud import bigquery
LOCATION = "us"

In [4]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

bqclient = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

In [13]:
required_test_df = pd.DataFrame()

for table_number in tqdm(range(55)):
    query_string = f"""
    SELECT * EXCEPT (AAAA, BB, CC, label)
    FROM `sunlit-analyst-309609.training_set.train_table_{table_number}`
    """
    
    df = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    
    # Metrics to calculate
    mod = abs(df).max()
    train_mean = df.mean()
    train_std = df.std()
    
    train_mean.to_csv(storage_mean, header=None, sep=",", mode="a", index=False)
    train_std.to_csv(storage_std, header=None, sep=",", mode="a", index=False)
    mod.to_csv(storage_mod, header=None, sep=",", mode="a", index=False)

100%|██████████| 55/55 [58:18<00:00, 63.61s/it]


In [14]:
assert np.loadtxt(storage_mean, delimiter=",").shape == (16500,)
assert np.loadtxt(storage_std, delimiter=",").shape == (16500,)
assert np.loadtxt(storage_mod, delimiter=",").shape == (16500,)

In [15]:
query_string = f"""
    SELECT * EXCEPT (AAAA)
    FROM `sunlit-analyst-309609.training_set.noisy_train_extra_params`
    """

df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)

mod = abs(df).max()
train_mean = df.mean()
train_std = df.std()

train_mean.to_csv(storage_mean, header=None, sep=",", mode="a", index=False)
train_std.to_csv(storage_std, header=None, sep=",", mode="a", index=False)
mod.to_csv(storage_mod, header=None, sep=",", mode="a", index=False)

In [16]:
assert np.loadtxt(storage_mean, delimiter=",").shape == (16506,)
assert np.loadtxt(storage_std, delimiter=",").shape == (16506,)
assert np.loadtxt(storage_mod, delimiter=",").shape == (16506,)